In [1]:
# 필요한 라이브러리 설치 (없는 경우에만 설치됨)
import sys
import subprocess

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# 필요한 패키지 리스트
packages = [
    "requests", 
    "beautifulsoup4", 
    "pandas", 
    "numpy"
]

# 패키지 설치
for package in packages:
    try:
        __import__(package)
        print(f"{package} 이미 설치되어 있습니다.")
    except ImportError:
        print(f"{package} 설치 중...")
        install_package(package)
        print(f"{package} 설치 완료!")
        
print("모든 필요한 라이브러리가 설치되었습니다.")

requests 이미 설치되어 있습니다.
beautifulsoup4 설치 중...


beautifulsoup4 설치 완료!
pandas 이미 설치되어 있습니다.
numpy 이미 설치되어 있습니다.
모든 필요한 라이브러리가 설치되었습니다.


In [5]:
# 필요한 라이브러리 임포트
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import os
import json
import urllib.parse

def get_news_content(url):
    """
    뉴스 URL에서 본문 내용을 추출하는 함수
    
    Args:
        url (str): 뉴스 기사 URL
        
    Returns:
        str: 추출된 뉴스 본문
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'
    }
    
    try:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 네이버 뉴스 본문 선택자
        content_element = soup.select_one("#dic_area")
        if content_element:
            # 불필요한 요소 제거
            for tag in content_element.select('.end_photo_org, script'):
                tag.decompose()
            content = content_element.get_text().strip()
            return re.sub(r'\s+', ' ', content).strip()
        
        # 다른 본문 선택자 시도
        content_element = soup.select_one("#newsct_article, .newsct_article, article")
        if content_element:
            content = content_element.get_text().strip()
            return re.sub(r'\s+', ' ', content).strip()
            
        return "본문을 추출할 수 없습니다."
    
    except Exception as e:
        return f"오류 발생: {str(e)}"

def scrape_naver_news_with_link(url):
    """
    직접 뉴스 URL에서 정보를 추출하는 함수
    
    Args:
        url (str): 뉴스 기사 URL
        
    Returns:
        dict: 뉴스 정보 딕셔너리
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'
    }
    
    try:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 제목 추출
        title_element = soup.select_one(".media_end_head_headline, h2.end_tit, h3.tit_headline, .sds-comps-text-type-headline1")
        title = title_element.get_text().strip() if title_element else "제목 없음"
        
        # 언론사 추출
        press_element = soup.select_one(".media_end_head_top_logo_link img, .press_logo img")
        press = press_element.get('alt', '') if press_element else ""
        
        if not press:
            press_element = soup.select_one(".media_end_head_top_logo_link, .press_logo")
            press = press_element.get_text().strip() if press_element else ""
        
        # 날짜 추출
        date_element = soup.select_one(".media_end_head_info_datestamp_time, .info_group .time, .article_info .date")
        date = date_element.get_text().strip() if date_element else ""
        
        # 본문 추출
        content_element = soup.select_one("#dic_area, #newsct_article")
        if content_element:
            # 불필요한 요소 제거
            for tag in content_element.select('.end_photo_org, script'):
                tag.decompose()
            
            content = content_element.get_text().strip()
            content = re.sub(r'\s+', ' ', content).strip()
        else:
            content = "본문을 추출할 수 없습니다."
        
        # 스니펫 생성
        snippet_element = soup.select_one(".sds-comps-text-type-body1, meta[property='og:description']")
        if snippet_element and snippet_element.name == 'meta':
            snippet = snippet_element.get('content', '')
        elif snippet_element:
            snippet = snippet_element.get_text().strip()
        else:
            snippet = content[:150] + "..." if len(content) > 150 else content
        
        return {
            'title': title,
            'link': url,
            'snippet': snippet,
            'press': press,
            'date': date,
            'content': content
        }
        
    except Exception as e:
        print(f"뉴스 추출 중 오류 발생: {e}")
        return None

def search_naver_news_by_keyword(keyword, limit=10):
    """
    네이버 뉴스에서 키워드로 검색하여 뉴스 목록을 가져오는 함수
    
    Args:
        keyword (str): 검색할 키워드
        limit (int): 검색할 최대 뉴스 개수
        
    Returns:
        DataFrame: 뉴스 정보가 담긴 데이터프레임
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'
    }
    
    news_list = []
    page = 1
    encoded_keyword = urllib.parse.quote(keyword)
    collected_count = 0
    
    print(f"키워드 '{keyword}'로 뉴스 검색 중...")
    
    while collected_count < limit:
        start = (page - 1) * 10 + 1
        search_url = f"https://search.naver.com/search.naver?where=news&query={encoded_keyword}&start={start}"
        
        try:
            print(f"검색 페이지 {page} 로딩 중...")
            response = requests.get(search_url, headers=headers)
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # 네이버 뉴스 링크 찾기
            links = soup.select("a.lu8Lfh20c9DvvP05mqBf, a[href*='news.naver.com']")
            
            if not links:
                print("더 이상 뉴스 링크를 찾을 수 없습니다.")
                break
                
            print(f"페이지 {page}에서 {len(links)}개의 뉴스 링크 발견")
            
            # 중복 없이 링크만 추출
            unique_links = set()
            for link in links:
                href = link.get('href', '')
                if 'news.naver.com' in href and href not in unique_links:
                    unique_links.add(href)
            
            print(f"중복 제거 후 {len(unique_links)}개의 고유 링크 발견")
            
            # 링크에서 뉴스 정보 추출
            for href in unique_links:
                if collected_count >= limit:
                    print(f"목표한 뉴스 개수({limit}개)에 도달했습니다.")
                    break
                    
                print(f"[{collected_count+1}/{limit}] 뉴스 정보 추출 중: {href}")
                news_info = scrape_naver_news_with_link(href)
                
                if news_info:
                    collected_count += 1
                    news_list.append(news_info)
                    print(f"  - 제목: {news_info['title']}")
                    
                    # 과도한 요청 방지
                    if collected_count < limit:
                        print("  - 다음 뉴스 처리 전 1초 대기...")
                        time.sleep(1)
            
            # 이미 충분한 뉴스를 수집했으면 전체 루프 중단
            if collected_count >= limit:
                print(f"목표한 뉴스 개수({limit}개)를 모두 수집했습니다.")
                break
                
            # 다음 페이지 확인 전 대기
            print(f"다음 페이지 검색 전 2초 대기 중...")
            time.sleep(2)
            page += 1
                
        except Exception as e:
            print(f"검색 중 오류 발생: {e}")
            import traceback
            traceback.print_exc()
            break
    
    print(f"총 {len(news_list)}개의 뉴스를 수집했습니다.")
    return pd.DataFrame(news_list)

def save_results(df, output_file, format='csv'):
    """
    결과를 저장하는 함수
    
    Args:
        df (DataFrame): 저장할 데이터프레임
        output_file (str): 출력 파일 경로
        format (str): 출력 형식 (csv 또는 json)
    """
    if not output_file:
        output_file = "news"
    
    # 확장자 확인 및 추가
    base, ext = os.path.splitext(output_file)
    if not ext:
        output_file = f"{output_file}.{format}"
    
    # 형식에 따라 저장
    if format == 'json' or output_file.endswith('.json'):
        df.to_json(output_file, orient='records', force_ascii=False, indent=4)
    else:
        df.to_csv(output_file, index=False)
    
    print(f"'{output_file}'로 결과가 저장되었습니다.")

# 사용자 입력 받기
keyword = input("검색할 키워드를 입력하세요: ").strip()
if not keyword:
    print("키워드가 입력되지 않았습니다. 기본값 '인공지능'으로 설정합니다.")
    keyword = "인공지능"

# 뉴스 개수 입력
limit_str = input("수집할 최대 뉴스 개수를 입력하세요 (기본값: 10): ").strip()
try:
    limit = int(limit_str) if limit_str else 10
    if limit <= 0:
        print("뉴스 개수는 1 이상이어야 합니다. 기본값 10로 설정합니다.")
        limit = 10
except ValueError:
    print("올바른 숫자가 아닙니다. 기본값 10로 설정합니다.")
    limit = 10

# 저장 형식 입력
format_str = input("저장 형식을 선택하세요 (csv 또는 json, 기본값: csv): ").strip().lower()
format_type = format_str if format_str in ['csv', 'json'] else 'csv'

# 출력 파일명 입력
output_file = input(f"저장할 파일명을 입력하세요 (기본값: {keyword}_news.{format_type}): ").strip()
if not output_file:
    output_file = f"{keyword}_news.{format_type}"

# 뉴스 수집 시작
print(f"\n키워드 '{keyword}'로 최대 {limit}개의 뉴스를 수집합니다...")
news_df = search_naver_news_by_keyword(keyword, limit)

if news_df.empty:
    print("뉴스를 찾을 수 없습니다.")
else:
    # 결과 저장
    save_results(news_df, output_file, format_type)
    
    # 뉴스 요약 출력
    print("\n====== 수집된 뉴스 ======")
    for i, (_, row) in enumerate(news_df.iterrows()):
        print(f"{i+1}. {row['title']} ({row['press']}, {row['date']})")
        print(f"   - 요약: {row['snippet']}")
        print(f"   - 링크: {row['link']}")
        print(f"   - 본문 일부: {row['content'][:100]}...")
        print("")
        
    # 결과 데이터프레임 확인
    display(news_df.head())


키워드 '인공지능'로 최대 10개의 뉴스를 수집합니다...
키워드 '인공지능'로 뉴스 검색 중...
검색 페이지 1 로딩 중...
페이지 1에서 67개의 뉴스 링크 발견
중복 제거 후 12개의 고유 링크 발견
[1/10] 뉴스 정보 추출 중: https://n.news.naver.com/mnews/article/001/0015357393?sid=102
  - 제목: 재판에 AI 기술 도입 논의한다…사법부 인공지능위원회 출범
  - 다음 뉴스 처리 전 1초 대기...
[2/10] 뉴스 정보 추출 중: https://news.naver.com/main/static/channelPromotion.html
  - 제목: 제목 없음
  - 다음 뉴스 처리 전 1초 대기...
[3/10] 뉴스 정보 추출 중: https://n.news.naver.com/mnews/article/366/0001073043?sid=101
  - 제목: NH투자증권, 인공지능이 투자해 주는 ‘퇴직연금 로보어드바이저 일임서비스’ 출시
  - 다음 뉴스 처리 전 1초 대기...
[4/10] 뉴스 정보 추출 중: https://n.news.naver.com/mnews/article/277/0005585026?sid=102
  - 제목: 'AI 기술' 재판 업무에 활용…법원행정처 산하 인공지능위원회 출범
  - 다음 뉴스 처리 전 1초 대기...
[5/10] 뉴스 정보 추출 중: https://n.news.naver.com/mnews/article/011/0004479462?sid=102
  - 제목: AI 재판 현실되나…대법 '사법부 인공지능위원회' 출범
  - 다음 뉴스 처리 전 1초 대기...
[6/10] 뉴스 정보 추출 중: https://n.news.naver.com/mnews/article/003/0013210449?sid=105
  - 제목: 기업 리더 3명 중 1명 "AI 비서 쓰면 인력 조정 고려할 것'"
  - 다음 뉴스 처리 전 1초 대기...
[7/10] 뉴스 정보 추출 중:

,title,link,snippet,press,date,content
0,재판에 AI 기술 도입 논의한다…사법부 인공지능위원회 출범,https://n.news.naver.com/mnews/article/001/001...,위원장 '정보기술 전문가' 이숙연 대법관…AI 기반 사법정보화 시스템 구축 목표 대...,,2025.04.28. 오후 6:06,위원장 '정보기술 전문가' 이숙연 대법관…AI 기반 사법정보화 시스템 구축 목표(서...
1,제목 없음,https://news.naver.com/main/static/channelProm...,본문을 추출할 수 없습니다.,,,본문을 추출할 수 없습니다.
2,"NH투자증권, 인공지능이 투자해 주는 ‘퇴직연금 로보어드바이저 일임서비스’ 출시",https://n.news.naver.com/mnews/article/366/000...,NH투자증권이 퇴직연금 로보어드바이저 일임서비스를 출시했다고 28일 밝혔다. 퇴직연...,,2025.04.28. 오후 4:35,NH투자증권이 퇴직연금 로보어드바이저 일임서비스를 출시했다고 28일 밝혔다.퇴직연금...
3,'AI 기술' 재판 업무에 활용…법원행정처 산하 인공지능위원회 출범,https://n.news.naver.com/mnews/article/277/000...,대법원 법원행정처가 인공지능(AI) 기술을 재판 업무에 활용하는 방안을 논의하는 위...,,2025.04.28. 오후 7:12,대법원 법원행정처가 인공지능(AI) 기술을 재판 업무에 활용하는 방안을 논의하는 위...
4,AI 재판 현실되나…대법 '사법부 인공지능위원회' 출범,https://n.news.naver.com/mnews/article/011/000...,대한민국 법원이 인공지능(AI) 기술을 활용해 재판을 보다 신속하고 공정하게 만들기...,,2025.04.28. 오후 6:29,재판 지원 AI 플랫폼 개발 본격화30일 ‘재판지원 AI 플랫폼’ 설명회 개최[서울...
